# Import python libraries

In [ ]:
import numpy as np
import pandas as pd
import pandas
from sklearn import preprocessing
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

# Arabic_tweets_positive dataset

In [ ]:
cols = ['class','tweet_text']
positive = pd.read_csv('train_Arabic_tweets_positive_20190413.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)

# Preprocessing positive data

In [ ]:
positive.head(5)

,class,tweet_text
0,pos,نحن الذين يتحول كل ما نود أن نقوله إلى دعاء لل...
1,pos,وفي النهاية لن يبقىٰ معك آحدإلا من رأىٰ الجمال...
2,pos,من الخير نفسه 💛
3,pos,#زلزل_الملعب_نصرنا_بيلعب كن عالي الهمه ولا ترض...
4,pos,الشيء الوحيد الذي وصلوا فيه للعالمية هو : المس...


* # cleaning tweet text

In [ ]:
for letter in '#.][!XR':
    positive['tweet_text'] = positive['tweet_text'].astype(str).str.replace(letter,'')

In [ ]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

* # remove repeating char

In [ ]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

* # ProcessPost for applying all functions

In [ ]:
def processPost(tweet):

    #Replace @username with empty string
    tweet = re.sub('@[^\s]+', ' ', tweet)

    #Convert www.* or https?://* to " "
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',tweet)

    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)

    # remove punctuations
    tweet= remove_punctuations(tweet)

    # remove repeated letters
    tweet=remove_repeating_char(tweet)

    return tweet

* # Applying processpost function for preprocessing

In [ ]:
positive["tweet_text"] = positive['tweet_text'].apply(lambda x: processPost(x)) # apply used to call the method processpost

* # Getting Tokenize the tweet text

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
positive["tweet_text"] = positive["tweet_text"].apply(tokenizer.tokenize)

* # Stop words

In [ ]:
stopwords_list = stopwords.words('arabic')

In [ ]:
stopwords_list

['إذ',
 'إذا',
 'إذما',
 'إذن',
 'أف',
 'أقل',
 'أكثر',
 'ألا',
 'إلا',
 'التي',
 'الذي',
 'الذين',
 'اللاتي',
 'اللائي',
 'اللتان',
 'اللتيا',
 'اللتين',
 'اللذان',
 'اللذين',
 'اللواتي',
 'إلى',
 'إليك',
 'إليكم',
 'إليكما',
 'إليكن',
 'أم',
 'أما',
 'أما',
 'إما',
 'أن',
 'إن',
 'إنا',
 'أنا',
 'أنت',
 'أنتم',
 'أنتما',
 'أنتن',
 'إنما',
 'إنه',
 'أنى',
 'أنى',
 'آه',
 'آها',
 'أو',
 'أولاء',
 'أولئك',
 'أوه',
 'آي',
 'أي',
 'أيها',
 'إي',
 'أين',
 'أين',
 'أينما',
 'إيه',
 'بخ',
 'بس',
 'بعد',
 'بعض',
 'بك',
 'بكم',
 'بكم',
 'بكما',
 'بكن',
 'بل',
 'بلى',
 'بما',
 'بماذا',
 'بمن',
 'بنا',
 'به',
 'بها',
 'بهم',
 'بهما',
 'بهن',
 'بي',
 'بين',
 'بيد',
 'تلك',
 'تلكم',
 'تلكما',
 'ته',
 'تي',
 'تين',
 'تينك',
 'ثم',
 'ثمة',
 'حاشا',
 'حبذا',
 'حتى',
 'حيث',
 'حيثما',
 'حين',
 'خلا',
 'دون',
 'ذا',
 'ذات',
 'ذاك',
 'ذان',
 'ذانك',
 'ذلك',
 'ذلكم',
 'ذلكما',
 'ذلكن',
 'ذه',
 'ذو',
 'ذوا',
 'ذواتا',
 'ذواتي',
 'ذي',
 'ذين',
 'ذينك',
 'ريث',
 'سوف',
 'سوى',
 'شتان',
 'عدا',
 'عسى',
 'عل'

In [ ]:
listToStr = ' '.join([str(elem) for elem in stopwords_list])

In [ ]:
listToStr

'إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هن

* # Removing stop words

In [ ]:
positive["tweet_text"]=positive["tweet_text"].apply(lambda x: [item for item in x if item not in stopwords_list])

# Arabic_tweets_negative dataset

In [ ]:
cols = ['class','tweet_text']
negative = pd.read_csv('train_Arabic_tweets_negative_20190413.tsv',sep='\t', error_bad_lines = False ,header=None, names=cols)

# Preprocessing negative data

In [ ]:
negative.head()

,class,tweet_text
0,neg,اعترف ان بتس كانو شوي شوي يجيبو راسي لكن اليوم...
1,neg,توقعت اذا جات داريا بشوفهم كاملين بس لي للحين ...
2,neg,#الاهلي_الهلال اكتب توقعك لنتيجة لقاء الهلال و...
3,neg,نعمة المضادات الحيوية . تضع قطرة💧مضاد بنسلين ع...
4,neg,الدودو جايه تكمل علي 💔


* # cleaning tweet text

In [ ]:
for letter in '#.][!XR':
    negative['tweet_text'] = negative['tweet_text'].astype(str).str.replace(letter,'')

* # applying processPost function for preprocessing

In [ ]:
negative["tweet_text"] = negative['tweet_text'].apply(lambda x: processPost(x)) # apply used to call the method processpost

* # Getting Tokenize the tweet text

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
negative["tweet_text"] = negative["tweet_text"].apply(tokenizer.tokenize)

* # Removing stop words

In [ ]:
negative["tweet_text"]=negative["tweet_text"].apply(lambda x: [item for item in x if item not in stopwords_list])

# Combining the positive and negative classes and tweets text

In [ ]:
final_data = pd.concat([positive, negative], axis=0)

In [ ]:
final_data.head()

,class,tweet_text
0,pos,"[يتحول, نود, نقوله, دعاء, تبحثوا, فينا, قوة, م..."
1,pos,"[وفي, النهاية, يبقى, معك, آحدإلا, الجمال, روحك..."
2,pos,"[الخير, نفسه]"
3,pos,"[زلزلالملعبنصرنابيلعب, عالي, الهمه, ترضى, بغير..."
4,pos,"[الشيء, الوحيد, وصلوا, لعالمية, المسيار, ترى, ..."


# Count of each target class

In [ ]:
y=final_data['class']
y.value_counts()

pos    22761
neg    22514
Name: class, dtype: int64

# Features Extraction from tweets text with TFIDF unigram

In [ ]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    max_features =10000)

unigramdataGet= word_vectorizer.fit_transform(final_data['tweet_text'].astype('str'))
unigramdataGet = unigramdataGet.toarray()


# encoding class as 1 for Postive class and 0 for negative class

In [ ]:
pro= preprocessing.LabelEncoder()
encpro=pro.fit_transform(final_data['class'])
final_data['class'] = encpro

# By getting features and Class

In [ ]:
y=final_data['class']
X=unigramdataGet

# Spliting Dataset into 70% Training and 30% Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

# --------Trainig and Testing with Machine Learning Algorithms ----------------

# Naive Bayes Algorithm

In [ ]:
nb=GaussianNB()
nb= nb.fit(X_train , y_train)
nb

GaussianNB()

#### Accuracy

In [ ]:
y_pred = nb.predict(X_test)
nb_1=nb.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(nb.score(X_test, y_test)))

Accuracy= 0.722


# PassiveAggressiveClassifier Algorithm

In [ ]:
PC= PassiveAggressiveClassifier()
PC= PC.fit(X_train , y_train)
PC

PassiveAggressiveClassifier()

#### Accuracy

In [ ]:
y_pred = PC.predict(X_test)
pc_1=PC.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(PC.score(X_test, y_test)))

Accuracy= 0.753


# LogisticRegression Algorithm

In [ ]:
LR= LogisticRegression()
LR= LR.fit(X_train , y_train)
LR

LogisticRegression()

#### Accuracy

In [ ]:
y_pred = LR.predict(X_test)
lr_1=LR.score(X_test, y_test)
print('Accuracy= {:.3f}'.format(LR.score(X_test, y_test)))

Accuracy= 0.755


# Comparison of all algorithms Results

In [ ]:
x = PrettyTable()
print('\n')
print("Comparison of all algorithms on F1 score")
x.field_names = ["Model", "Accuracy"]


x.add_row(["Naive Bayes Algorithm", round(nb_1,2)])
x.add_row(["Passive Aggressive Classifier Algorithm", round(pc_1,2)])
x.add_row(["Logistic Regression Algorithm", round(lr_1,2)])

print(x)
print('\n')



Comparison of all algorithms on F1 score
+-----------------------------------------+----------+
|                  Model                  | Accuracy |
+-----------------------------------------+----------+
|          Naive Bayes Algorithm          |   0.72   |
| Passive Aggressive Classifier Algorithm |   0.75   |
|      Logistic Regression Algorithm      |   0.75   |
+-----------------------------------------+----------+




# Training Logistic Regression

## now we know that is giving better results so we can use the final model application as well

In [ ]:
RC =LogisticRegression()
RC.fit(X,y)

LogisticRegression()

# Prediction on sample text

In [ ]:
tv=["ما احلى هذا الاسايمنت"]

In [ ]:
x = word_vectorizer.transform(tv)

In [ ]:
pred=RC.predict(x)
pred=pro.inverse_transform(pred)
prediction=pd.DataFrame(pred, columns=['Prediction'])
print (prediction)

  Prediction
0        pos


In [ ]:
tv=["ما اسوء هذا المنتج"]

In [ ]:
x = word_vectorizer.transform(tv)

In [ ]:
pred=RC.predict(x)
pred=pro.inverse_transform(pred)
prediction=pd.DataFrame(pred, columns=['Prediction'])
print (prediction)

  Prediction
0        neg
